In [1]:
%run "../check_setup.ipynb"

SAP HANA Client for Python: 2.23.25010300
Connected to SAP HANA db version 4.00.000.00.1737457831 (fa/CE2024.40) 
at c5889dd5-e0f6-4930-8408-94d53ca61dbf.hna0.prod-us10.hanacloud.ondemand.com:443 as CODEJAMHANAAI00
Current time on the SAP HANA server: 2025-01-30 19:43:59.863000


In [2]:
import requests, os

os.environ["USER_AGENT"] = f"{requests.get('https://httpbin.org/user-agent').json()['user-agent']}"
os.environ["USER_AGENT"]

'python-requests/2.32.3'

In [4]:
from loader_sapblog import CustomSAPBlogLoader

# Define URLs
urls = [
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177423", # "New Machine Learning and NLP features in SAP HANA Cloud 2024 Q4"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177557", # "Text Chunking – An Exciting New NLP Function in SAP HANA Cloud"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58847", # "New Text Analysis in SAP HANA Cloud Predictive Analysis Library (PAL)"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58846", # "Text Embedding Service in SAP HANA Cloud Predictive Analysis Library (PAL)"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177553", # "New Information Retrieval Techniques in SAP HANA Cloud using BM25 and ANNS for Advanced Text Mining"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58845", # "Dimensionality Reduction of Text Embeddings for Hybrid Prediction Data"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58419", # "Document Clustering using KMeans and Text Embeddings"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/176747", # "Hybrid Prediction with Tabular and Text Inputs using Hybrid Gradient Boosting Trees"
    "https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177560", # "Exploring ML Explainability in SAP HANA PAL - AutoML"

]

# Load and process articles
loader = CustomSAPBlogLoader(urls)
(documents := loader.load()).__len__()


9

In [6]:

# Check the length of each article's content
for doc in documents:
    print(doc.metadata["source"], len(doc.page_content), end="\n")


https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177423 20475
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177557 12979
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58847 12891
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58846 17156
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177553 25245
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58845 14210
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/erp-blog-sap/article-id/58419 5867
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/176747 7691
https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177560 18610


In [7]:
print(documents[0].page_content[:300])

With the SAP HANA Cloud 2024 Q4 release, several new embedded **Machine Learning / AI and NLP functions** have been released with the SAP HANA Cloud **Predictive Analysis Library (PAL)**. Key new capabilities to be highlighted are introduction of **text analysis**, **text vectorization**, **vector d


In [8]:
connection_to_hana = myconn.connection

In [10]:
from langchain_community.vectorstores.hanavector import HanaDB
from embedder_saphana import HanaEmbeddings

db = HanaDB(
    embedding=HanaEmbeddings(connection_to_hana), connection=connection_to_hana, table_name="MY_LANGCHAIN_EMBEDDINGS"
)

print(f"Table to be used is {db.table_name}")

Table to be used is MY_LANGCHAIN_EMBEDDINGS


In [11]:
db.delete(filter={})

True

In [12]:
%%time
db.add_documents(documents=documents)

CPU times: user 20.9 ms, sys: 480 μs, total: 21.4 ms
Wall time: 1.68 s


[]

## Text splitting, preparing articles's text for vectorization

In [15]:
hdf_langchain_tab = myconn.table(db.table_name).add_id()

In [16]:
hdf_langchain_tab.get_table_structure()

{'ID': 'INT',
 'VEC_TEXT': 'NCLOB',
 'VEC_META': 'NCLOB',
 'VEC_VECTOR': 'REAL_VECTOR'}

In [20]:
import pandas as pd
pd.set_option('max_colwidth', 128) 

hdf_langchain_tab.head(1).collect().T

,0
ID,1
VEC_TEXT,"With the SAP HANA Cloud 2024 Q4 release, several new embedded **Machine Learning / AI and NLP functions** have been released..."
VEC_META,"{""source"": ""https://community.sap.com/t5/blogs/blogarticleprintpage/blog-id/technology-blog-sap/article-id/177423""}"
VEC_VECTOR,"[-0.023911751806735992, -0.028109241276979446, 0.01552630215883255, 0.013691979460418224, 0.04396688938140869, 0.00687683559..."


In [24]:
# Applying the Text Splitter with recursive-splitting, available with hana-ml 2.23
from hana_ml.text.text_splitter import TextSplitter

splitter = TextSplitter(split_type='recursive', chunk_size=1024, overlap=64)
#splitter._extend_pal_parameter({'GLOBAL_SEPARATOR':'[. ]', 'KEEP_SEPARATOR':1})
splitted_text = splitter.split_text(
    hdf_langchain_tab.select('ID', 'VEC_TEXT'), 
    order_status=True
    )

In [25]:
import pandas as pd
pd.set_option('max_colwidth', None) 

print(splitted_text.shape)
display(splitter.statistics_.collect())

display(splitted_text.select("*", ('LENGTH("CONTENT")', "CHUNCK_SIZE")).head(10).collect())

[177, 3]


,STAT_NAME,STAT_VALUE
0,GLOBAL_SEPARATOR_LIST,"{""Separator"":""[\n\n,\n, ]""}"


,ID,SUB_ID,CONTENT,CHUNCK_SIZE
0,1,0,"With the SAP HANA Cloud 2024 Q4 release, several new embedded **Machine Learning / AI and NLP functions** have been released with the SAP HANA Cloud **Predictive Analysis Library (PAL)**. Key new capabilities to be highlighted are introduction of **text analysis**, **text vectorization**, **vector data as input** to many more Machine Learning functions like **classification** and **regression** in the SAP HANA Cloud database. An enhancement summary is available in the What’s new document for [SAP HANA Cloud database 2024.40 (QRC 4/2024)](https://help.sap.com/whats-new/2495b34492334456a49084831c2bea4e?Category=Predictive+Analysis+Library&Valid_as_Of=2024-12-01:2024-12-30&locale=en-US).\n![ChristophMorgen_0-1733594472111.png](/t5/image/serverpage/image-id/199433i6EC88FC9CBCC847F/image-size/large?v=v2&px=999 ""ChristophMorgen_0-1733594472111.png"")\n## Text analysis and text vectorization\n### Text chunking",912
1,1,1,"## Text analysis and text vectorization\n### Text chunking\nText data or documents, for further analysis typically requires to be broken down into smaller, more manageable segments such as sentences, paragraphs, or semantic units. This facilitates improved subsequent natural language processing (NLP) tasks. Moreover, processing text units by embedding models and Large Language Models (LLMs) could result in information loss or inaccuracies, as these models often have text token length limits for processing. Hence, text chunking ensures efficient text processing, mitigating potential issues.\nA new [text chunking](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-predictive-analysis-library/text-chunking-text-chunking?) function is introduced in the Predictive Analysis Library (PAL), which supports multiple splitting methods\n* Splitting by character\n* Recursive splitting by a list of separators or words\n* Splitting by document type or language",981
2,1,2,"* Splitting by document type or language\nFor **text chunking** a SQL function as well as new method in the Python ML client is made available:\n![ChristophMorgen_1-1733594472114.png](/t5/image/serverpage/image-id/199432i8B4ADD10CD68A12D/image-size/large?v=v2&px=999 ""ChristophMorgen_1-1733594472114.png"")\nThe new text splitting function are explained in more detail in the following blog post: [Text chunking - an exciting new NLP function in SAP HANA Cloud](https://community.sap.com/t5/technology-blogs-by-sap/text-chunking-an-exciting-new-nlp-function-in-sap-hana-cloud/ba-p/13958766), by [@xinchen](/t5/user/viewprofilepage/user-id/712820) .\n### Text analysis\nA key technique in **Natural Language Processing (NLP)** is **text analysis**, aiming to automatically extract information like a sentiment expressed or detected entities from unstructured text data.",862
3,1,3,"In SAP HANA Cloud a new [text analysis](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-predictive-analysis-library/text-analysis-textanalysis?state=DRAFT) function within the Predictive Analysis Library is now introduced, which utilizes tiny BERT NLP models to process the text data for the following tasks\n* **Named Entity Recognition (NER)**, for identifying entities like names of persons, organizations, locations, time expressions, etc.,\n* **Sentiment****analysis,** for determining the expressed positive, negative, or neutral sentiment in a document, sentence of text phrase,\n* **Part-of-Speech (POS) tagging,** for identifying the grammatical category or words like nouns, verbs, adjectives, and adverbs. It is essential for comprehending the syntactic structure of languages, and can benefit many further NLP scenarios like two text analysis tasks mentioned above, i.e. NER and sentiment analysis",937
4,1,4,"The text analysis task to be applied for each text object, can be specified using the task-column (e.g. ‘pos, ner, sentiment-phrase-score’) of the input table to the PAL\_TEXT\_Analysis function:\n

## Generating Text Embeddings in SAP HANA Cloud

In [26]:
print(f"""Number of records selected for further processing: {splitted_text.count()}""")

Number of records selected for further processing: 177


In [27]:
### Generating Text Embeddings in SAP HANA Cloud with the new PAL function, function available with hana-ml 2.23.
from hana_ml.text.pal_embeddings import PALEmbeddings
pe = PALEmbeddings()
textembeddings = pe.fit_transform(splitted_text, key="SUB_ID", target=["CONTENT"], thread_number=10, batch_size=10) #, max_token_num=512
print(f"{textembeddings.count()} records processed in {round(pe.runtime, 3)} sec")

177 records processed in 3.658 sec


In [28]:
textembeddings.get_table_structure()

{'ID': 'INT',
 'SUB_ID': 'INT',
 'CONTENT': 'NCLOB',
 'VECTOR_COL_CONTENT': 'REAL_VECTOR'}

In [29]:
textembeddings.head().collect()

,ID,SUB_ID,CONTENT,VECTOR_COL_CONTENT
0,1,0,"With the SAP HANA Cloud 2024 Q4 release, several new embedded **Machine Learning / AI and NLP functions** have been released with the SAP HANA Cloud **Predictive Analysis Library (PAL)**. Key new capabilities to be highlighted are introduction of **text analysis**, **text vectorization**, **vector data as input** to many more Machine Learning functions like **classification** and **regression** in the SAP HANA Cloud database. An enhancement summary is available in the What’s new document for [SAP HANA Cloud database 2024.40 (QRC 4/2024)](https://help.sap.com/whats-new/2495b34492334456a49084831c2bea4e?Category=Predictive+Analysis+Library&Valid_as_Of=2024-12-01:2024-12-30&locale=en-US).\n![ChristophMorgen_0-1733594472111.png](/t5/image/serverpage/image-id/199433i6EC88FC9CBCC847F/image-size/large?v=v2&px=999 ""ChristophMorgen_0-1733594472111.png"")\n## Text analysis and text vectorization\n### Text chunking","[-0.023911774158477783, -0.028109241276979446, 0.015526297502219677, 0.013691983185708523, 0.04396688938140869, 0.006876830942928791, 0.03683269023895264, -0.0035757615696638823, -0.06305049359798431, 0.009632617235183716, -0.021553143858909607, 0.004323621746152639, -0.027989041060209274, 0.025459082797169685, 0.03479696810245514, 0.028946630656719208, 0.027081387117505074, 0.003150323638692498, 0.07960813492536545, 0.026996217668056488, -0.010766889899969101, 0.0016619986854493618, 0.04091837629675865, -0.0028601456433534622, 0.0016500973142683506, -0.008816101588308811, 1.6541744116693735e-05, 0.0015323148109018803, -0.06911253184080124, -0.018324455246329308, 0.035987094044685364, -5.23844501003623e-05, -0.026142774149775505, -0.01637185923755169, 0.009819632396101952, 0.04864133894443512, -0.036709774285554886, 0.0032759925816208124, -0.018378810957074165, -0.023431824520230293, -0.05680566281080246, 0.03835161030292511, -0.02340955287218094, 0.0018498275894671679, -0.00692454120144248, 0.020993608981370926, -0.10551895946264267, 0.019862355664372444, 0.007885497063398361, 0.0009694462642073631, -0.08036550134420395, 0.004181185737252235, -0.032536301761865616, -0.025150753557682037, -0.0010078886989504099, 0.02551381103694439, -0.03170022368431091, -0.056162070482969284, -0.016596633940935135, -0.01743553951382637, 0.029681053012609482, -0.028969570994377136, -0.020978808403015137, -0.01587980054318905, 0.040055837482213974, -0.06240379810333252, -0.027329791337251663, -0.0038561318069696426, -0.03743898496031761, -0.029643619433045387, -0.01701352372765541, -0.0006565761286765337, -0.020680518820881844, -0.0036640872713178396, 0.00343317911028862, -0.053119562566280365, -0.0036951431538909674, 0.031123574823141098, 0.04208618029952049, 0.010570920072495937, 0.037269677966833115, 0.011130962520837784, 0.032465752214193344, 0.03077493980526924, -0.008563791401684284, -0.01373370923101902, -0.012327004224061966, -0.048215027898550034, -0.022186260670423508, 0.05310133844614029, 0.016308974474668503, -0.04790455847978592, 0.0607713982462883, 0.011412310414016247, 0.0002739897754509002, 0.04081522300839424, 0.015716614201664925, -0.01510055735707283, 0.00749729061499238, -0.02972254529595375, ...]"


In [30]:
prompt="What is BM25?"

In [31]:
myconn.sql(
    f"""SELECT 
    COSINE_SIMILARITY(VECTOR_EMBEDDING('{prompt}', 'QUERY', 'SAP_NEB.20240715'), "VECTOR_COL_CONTENT") AS "SIMILARITY", 
    L2DISTANCE(VECTOR_EMBEDDING('What{prompt}', 'QUERY', 'SAP_NEB.20240715'), "VECTOR_COL_CONTENT") AS "DISTANCE", 
    "ID", "SUB_ID", "CONTENT" 
    FROM ({textembeddings.select_statement})
    ORDER BY 1 DESC;
    """
).collect()

,SIMILARITY,DISTANCE,ID,SUB_ID,CONTENT
0,0.656120,0.791497,5,89,"## 2.2 BM25\nThe [BM25](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-predictive-analysis-library/bm25-search) (Best Matching 25) is an extension of the Binary Independence Model (BIM) and you could invoke a SQL procedure named PAL\_SEARCH\_DOCS\_BY\_KEYWORDS to use this algorithm. This procedure draws on the TFIDF (Term Frequency-Inverse Document Frequency) values calculated using the [PAL\_TF\_ANALYSIS](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-predictive-analysis-library/term-analysis-term-analysis-0dd1876). These TFIDF values are used to search for documents that closely align with user-provided keywords. Although there are a few variations of BM25 in existence, the one currently implemented is the original version."
1,0.607794,0.837420,5,83,"Simultaneously, BM25, as a representative of sparse information retrieval, optimizes the relevance scoring between documents and queries through its probabilistic framework. BM25 introduces saturation functions and empirically observed parameter adjustments, providing a more nuanced relevance score, which has led to its widespread adoption in search engines for ranking search results.\nBM25 is renowned for its simplicity and speed in handling large datasets but may struggle with semantic gaps between queries and documents. In contrast, ANN search offers better recall but at the cost of higher computational resources. The choice between BM25 and ANN search hinges on the application's requirements for efficiency, precision, and semantic depth."
2,0.607242,0.836186,5,97,"```\n## 3.2 BM25\nIf you wish to use BM25, in the hana-ml Python client , you should call the function [search\_docs\_by\_keywords()](https://help.sap.com/doc/cd94b08fe2e041c2ba778374572ddba9/2024_4_QRC/en-US/text/hana_ml.text.tm.search_docs_by_keywords.html#hana_ml.text.tm.search_docs_by_keywords). For SQL, the corresponding procedure is named PAL\_SEARCH\_DOCS\_BY\_KEYWORDS."
3,0.576916,0.880779,1,9,"### Keyword-based text search\nInformation retrieval from text typically starts with text search techniques. A new keyword-based search function [BM25 search](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-predictive-analysis-library/bm25-search?) (SEARCH\_DOCS\_BY\_KEYWORDS) is provided with the SAP HANA Cloud Predictive Analysis Library (PAL). BM25, is a widely used ranking functions in text search, it estimates the relevance of documents compared to a list of search keywords provided. It builds on top of term-/document-frequencies (TF\_IDF values).\n![ChristophMorgen_5-1733594472200.png](/t5/image/serverpage/image-id/199436i118C151CA67C282D/image-size/large?v=v2&px=999 ""ChristophMorgen_5-1733594472200.png"")"
4,0.571052,0.871983,5,85,"# 2. ANN Search and BM25 in SAP HANA Cloud PAL\n## 2.1 ANN Search\nThe [ANN search](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-predictive-analysis-library/no-content-for-file-anns-md) implementation in PAL leverages the Flat IVF algorithm, which clusters the original vector set and divides the vector space into smaller groups referred to as ""cells"". It then lists the reference of vectors belonging to each cluster for retrieval. KMeans clustering is used to build the vector-cluster references on a given dataset, dividing it into numerous clusters controlled by the parameter ""GROUP\_NUMBER"". These cluster centers are stored as the Vector-cluster References along with the original datasets into stateful in-memory ANN search model, thereby enabling much faster responses for search predictions."
...,...,...,...,...,...
172,0.337545,1.089470,2,31,"1. Character splitter : Splits the text based on a predefined number of characters, without considering whether it divides a complete word.\n2. Recursive splitter (by default) : Operates on a list of separators.\n3. Document splitter : Uses various